## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
# Import other libraries if needed

## Import Dataset

In [2]:
df_train = pd.read_csv('https://drive.google.com/uc?id=1RqoINbiatAf5ssVrqESn2F3vGCe_XY17')
df_train.head()

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1


In [3]:
df_test = pd.read_csv('https://drive.google.com/uc?id=1O9Fd-FRnJmQwGsqnUVQmAj8vYJM0NWH_')

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [26]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
train_set, val_set = train_test_split(df_train,test_size=0.25, random_state=42)

# check
print(train_set.shape)
print(val_set.shape)

(105303, 56)
(35101, 56)


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [5]:
# Mendapatkan informasi missing data in columns
train_set.isna().sum() / len(train_set) * 100

id                             0.000000
FILENAME                      40.849738
URL                           30.842426
URLLength                     43.275120
Domain                        49.959640
DomainLength                  32.906945
IsDomainIP                    29.968757
TLD                           32.335261
CharContinuationRate          34.234542
TLDLegitimateProb             37.853622
URLCharProb                   37.188874
TLDLength                     34.043664
NoOfSubDomain                 31.556556
HasObfuscation                46.707121
NoOfObfuscatedChar            47.531409
ObfuscationRatio              46.036675
NoOfLettersInURL              45.083236
LetterRatioInURL              46.841970
NoOfDegitsInURL               41.797480
DegitRatioInURL               38.167004
NoOfEqualsInURL               43.867696
NoOfQMarkInURL                31.371376
NoOfAmpersandInURL            32.342858
NoOfOtherSpecialCharsInURL    33.823348
SpacialCharRatioInURL         44.700531


Teknik handle Missing data
- replace missing value dengan nilai modus dari columns

In [6]:
imputer = SimpleImputer(strategy='most_frequent')

train_set = pd.DataFrame(imputer.fit_transform(train_set), columns=df_train.columns)

#mengubah kembali datatype menjadi semua (dari object)
for col in df_train.columns:
    train_set[col] = train_set[col].astype(df_train[col].dtype)

train_set

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,6208,100000.txt,https://www.farmranchstore.com,25.0,ipfs.io,18.0,0.0,com,1.0,0.522907,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1
1,160974,92017.txt,https://www.comlaude.com,25.0,www.comlaude.com,16.0,0.0,com,1.0,0.522907,...,0.0,0.0,1.0,30.0,2.0,0.0,143.0,0.0,0.0,1
2,174395,489814.txt,http://test-mantenimiento-bancaweb.azurewebsit...,23.0,ipfs.io,16.0,0.0,net,1.0,0.038420,...,0.0,0.0,1.0,4.0,2.0,8.0,0.0,0.0,0.0,1
3,39373,100000.txt,https://www.citygrab.co.uk,25.0,www.citygrab.co.uk,18.0,0.0,uk,1.0,0.522907,...,0.0,0.0,1.0,15.0,2.0,1.0,20.0,0.0,22.0,1
4,40047,100000.txt,https://www.chinacdc.cn,22.0,www.chinacdc.cn,18.0,0.0,com,1.0,0.003322,...,0.0,0.0,1.0,0.0,7.0,0.0,0.0,3.0,178.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105298,185151,100000.txt,https://www.gemathis.com,23.0,www.gemathis.com,16.0,0.0,com,1.0,0.522907,...,0.0,0.0,1.0,0.0,2.0,0.0,106.0,7.0,0.0,1
105299,201338,627933.txt,http://test-mantenimiento-bancaweb.azurewebsit...,24.0,www.mural24.co.uk,18.0,0.0,uk,0.8,0.522907,...,1.0,0.0,1.0,166.0,3.0,7.0,0.0,1.0,0.0,1
105300,174060,561645.txt,http://test-mantenimiento-bancaweb.azurewebsit...,34.0,ipfs.io,27.0,0.0,com,1.0,0.522907,...,1.0,0.0,1.0,0.0,2.0,0.0,42.0,0.0,25.0,1
105301,221529,100000.txt,https://www.leinsterrugby.ie,27.0,ipfs.io,20.0,0.0,ie,1.0,0.001588,...,0.0,0.0,1.0,0.0,2.0,57.0,0.0,0.0,331.0,1


In [7]:
train_set.columns

Index(['id', 'FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength',
       'IsDomainIP', 'TLD', 'CharContinuationRate', 'TLDLegitimateProb',
       'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation',
       'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL',
       'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',
       'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',
       'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS',
       'LineOfCode', 'LargestLineLength', 'HasTitle', 'Title',
       'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon', 'Robots',
       'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect', 'HasDescription',
       'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit', 'HasSocialNet',
       'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay',
       'Crypto', 'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS',
       'NoOfSelfRef', 'NoOfEmptyRef', 'NoOfExternalRef', 'label'],

In [8]:
train_set.isna().sum()

id                            0
FILENAME                      0
URL                           0
URLLength                     0
Domain                        0
DomainLength                  0
IsDomainIP                    0
TLD                           0
CharContinuationRate          0
TLDLegitimateProb             0
URLCharProb                   0
TLDLength                     0
NoOfSubDomain                 0
HasObfuscation                0
NoOfObfuscatedChar            0
ObfuscationRatio              0
NoOfLettersInURL              0
LetterRatioInURL              0
NoOfDegitsInURL               0
DegitRatioInURL               0
NoOfEqualsInURL               0
NoOfQMarkInURL                0
NoOfAmpersandInURL            0
NoOfOtherSpecialCharsInURL    0
SpacialCharRatioInURL         0
IsHTTPS                       0
LineOfCode                    0
LargestLineLength             0
HasTitle                      0
Title                         0
DomainTitleMatchScore         0
URLTitle

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [29]:
numerical_train_set = train_set.select_dtypes(include='number').columns
for col in numerical_train_set :
    print(f'min {col} : {train_set[col].min()}')
print('\n')
for col in numerical_train_set :
    print(f'max {col} : {train_set[col].max()}')

min id : 4
min URLLength : 14.0
min DomainLength : 4.0
min IsDomainIP : 0.0
min CharContinuationRate : 0.0
min TLDLegitimateProb : 0.0
min URLCharProb : 0.001229244
min TLDLength : 2.0
min NoOfSubDomain : 0.0
min HasObfuscation : 0.0
min NoOfObfuscatedChar : 0.0
min ObfuscationRatio : 0.0
min NoOfLettersInURL : 0.0
min LetterRatioInURL : 0.0
min NoOfDegitsInURL : 0.0
min DegitRatioInURL : 0.0
min NoOfEqualsInURL : 0.0
min NoOfQMarkInURL : 0.0
min NoOfAmpersandInURL : 0.0
min NoOfOtherSpecialCharsInURL : 0.0
min SpacialCharRatioInURL : 0.0
min IsHTTPS : 0.0
min LineOfCode : 2.0
min LargestLineLength : 23.0
min HasTitle : 0.0
min DomainTitleMatchScore : 0.0
min URLTitleMatchScore : 0.0
min HasFavicon : 0.0
min Robots : 0.0
min IsResponsive : 0.0
min NoOfURLRedirect : 0.0
min NoOfSelfRedirect : 0.0
min HasDescription : 0.0
min NoOfPopup : 0.0
min NoOfiFrame : 0.0
min HasExternalFormSubmit : 0.0
min HasSocialNet : 0.0
min HasSubmitButton : 0.0
min HasHiddenFields : 0.0
min HasPasswordField

In [ ]:
numerical_train_set = train_set.select_dtypes(include='number').columns
q1 = []
q3 = []
for i in numerical_train_set:
  Q1 = train_set[i].quantile(0.25)
  Q3 = train_set[i].quantile(0.75)

  q1.append(Q1)
  q3.append(Q3)

  IQR = Q3-Q1

  threshold = 1.5

  lowerBound = Q1 - (threshold*IQR)
  upperBound = Q3 + (threshold*IQR)

  # mean_value = train_set[i].mean()
  # train_set[i] = train_set[i].apply(lambda x: mean_value if (x < lowerBound or x > upperBound) else x)
  train_set[i] = train_set[i].clip(lower=lowerBound, upper=upperBound)

print(numerical_train_set)
print(q1)
print(q3)


Index(['id', 'URLLength', 'DomainLength', 'IsDomainIP', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon',
       'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect',
       'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit',
       'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields',
       'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo',
       'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef',
       'NoOfExternalRef', 'label'],
      dtype='object')
[np.float64(58791.0), 

In [46]:
numerical_train_set = train_set.select_dtypes(include='number').columns
for col in numerical_train_set :
    print(f'min {col} : {train_set[col].min()}')
print('\n')
for col in numerical_train_set :
    print(f'max {col} : {train_set[col].max()}')

min id : 4.0
min URLLength : 14.0
min DomainLength : 7.0
min IsDomainIP : 0.0
min CharContinuationRate : 0.8
min TLDLegitimateProb : 0.0
min URLCharProb : 0.04590618312500002
min TLDLength : 2.0
min NoOfSubDomain : 1.0
min HasObfuscation : 0.0
min NoOfObfuscatedChar : 0.0
min ObfuscationRatio : 0.0
min NoOfLettersInURL : 1.0
min LetterRatioInURL : 0.217
min NoOfDegitsInURL : 0.0
min DegitRatioInURL : 0.0
min NoOfEqualsInURL : 0.0
min NoOfQMarkInURL : 0.0
min NoOfAmpersandInURL : 0.0
min NoOfOtherSpecialCharsInURL : 1.0
min SpacialCharRatioInURL : 0.018
min IsHTTPS : 1.0
min LineOfCode : 2.0
min LargestLineLength : 23.0
min HasTitle : 1.0
min DomainTitleMatchScore : 0.0
min URLTitleMatchScore : 0.0
min HasFavicon : 0.0
min Robots : 0.0
min IsResponsive : 1.0
min NoOfURLRedirect : 0.0
min NoOfSelfRedirect : 0.0
min HasDescription : 0.0
min NoOfPopup : 0.0
min NoOfiFrame : 0.0
min HasExternalFormSubmit : 0.0
min HasSocialNet : 0.0
min HasSubmitButton : 0.0
min HasHiddenFields : 0.0
min Ha

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [12]:
# Write your code here
print(train_set.shape)
train_set = train_set.drop_duplicates(subset=["URL"])
print(train_set.shape)

(105303, 56)
(72823, 56)


### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [13]:
# Write your code here


## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [14]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded


### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [15]:
numerical_train_set = train_set.select_dtypes(include=['number']).columns

scaler = MinMaxScaler()

train_set[numerical_train_set] = scaler.fit_transform(train_set[numerical_train_set])

train_set

C:\Users\lenovo\AppData\Local\Temp\ipykernel_16000\1241875129.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set[numerical_train_set] = scaler.fit_transform(train_set[numerical_train_set])


,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,0.026311,100000.txt,https://www.farmranchstore.com,0.375,ipfs.io,0.4375,0.0,com,1.0,1.000000,...,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.000000,0.0,0.00000,1.0
1,0.682681,92017.txt,https://www.comlaude.com,0.375,www.comlaude.com,0.3125,0.0,com,1.0,1.000000,...,0.0,0.0,1.0,0.389610,0.166667,0.000000,0.608511,0.0,0.00000,1.0
2,0.739600,489814.txt,http://test-mantenimiento-bancaweb.azurewebsit...,0.125,ipfs.io,0.3125,0.0,net,1.0,0.073474,...,0.0,0.0,1.0,0.051948,0.166667,0.228571,0.000000,0.0,0.00000,1.0
3,0.166966,100000.txt,https://www.citygrab.co.uk,0.375,www.citygrab.co.uk,0.4375,0.0,uk,1.0,1.000000,...,0.0,0.0,1.0,0.194805,0.166667,0.028571,0.085106,0.0,0.22000,1.0
4,0.169824,100000.txt,https://www.chinacdc.cn,0.000,www.chinacdc.cn,0.4375,0.0,com,1.0,0.006353,...,0.0,0.0,1.0,0.000000,0.583333,0.000000,0.000000,1.0,0.39498,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105295,0.622068,100000.txt,https://www.macmegasite.com,0.375,ipfs.io,0.4375,0.0,com,1.0,1.000000,...,0.0,1.0,1.0,0.000000,0.166667,0.314286,0.000000,0.0,0.39498,1.0
105297,0.389167,775387.txt,https://www.cathedral.ca,0.125,ipfs.io,0.3125,0.0,ca,1.0,1.000000,...,0.0,0.0,1.0,0.272727,0.534936,0.265444,0.000000,1.0,0.00000,1.0
105298,0.785217,100000.txt,https://www.gemathis.com,0.125,www.gemathis.com,0.3125,0.0,com,1.0,1.000000,...,0.0,0.0,1.0,0.000000,0.166667,0.000000,0.451064,1.0,0.00000,1.0
105301,0.939497,100000.txt,https://www.leinsterrugby.ie,0.625,ipfs.io,0.5625,0.0,ie,1.0,0.003036,...,0.0,0.0,1.0,0.000000,0.166667,0.265444,0.000000,0.0,0.39498,1.0


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [16]:
# Write your code here

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [17]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [18]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [19]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [20]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [21]:
# Type your code here

## B. Naive Bayes

In [22]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [23]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [24]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`